In [167]:
# imports
import requests
from bs4 import BeautifulSoup
import pandas as pd
import math

In [168]:
# get html
response = requests.get('https://worldpopulationreview.com/countries/countries-by-gdp')

#print status code
print('Status code:', response.status_code, end= '\n')

Status code: 200


In [169]:
#Create soup object
html = response.content
html = BeautifulSoup(html, 'html.parser') 

# display markup 
#html.prettify()

# selecting element
gdp_data_table = html.find(name='table', attrs={'class' : 'jsx-364991517 table is-striped is-hoverable is-fullwidth tp-table-body is-narrow'})
gdp_data_row = gdp_data_table.find(name='tbody', attrs={'class':'jsx-364991517'}).find_all(name = 'tr')

#print html
# gdp_data_row


In [170]:
# creates empty list
gdp_datapoints = list()

#Extract data
for row in gdp_data_row:
    data = row.find_all('td')

    data = [td.contents[0] if len(td.contents) == 1 else 'NaN' for td in data] 
    data[1] = data[1].contents[0]
        
    country, gdp_imf_20, gdp_un_16, gdp_per_capita, population = data[1:]

    gdp_data = {
        "country" : country,
        "gdp(imf '20)" : gdp_imf_20,
        "gdp per capita" : gdp_per_capita,
        "population" : population
    } 

    gdp_datapoints.append(gdp_data)


# print(gdp_datapoints)

    

In [177]:
# create dataframe
df_gdp = pd.DataFrame(gdp_datapoints)
df_gdp.index = df_gdp.index + 1

df_gdp.head(15)

,country,gdp(imf '20),gdp per capita,population
1,United States,20.89 Tn,"$61,763","338,289,857"
2,China,14.86 Tn,"$10,423","1,425,887,337"
3,Japan,5.04 Tn,"$40,662","123,951,692"
4,Germany,3.84 Tn,"$46,100","83,369,843"
5,United Kingdom,2.76 Tn,"$40,867","67,508,936"
6,India,2.67 Tn,"$1,882","1,417,173,173"
7,France,2.62 Tn,"$40,571","64,626,628"
8,Italy,1.89 Tn,"$32,031","59,037,474"
9,Canada,1.65 Tn,"$42,789","38,454,327"
10,South Korea,1.64 Tn,"$31,617","51,815,810"


In [172]:
#get columns
columns = list()
for column in df_gdp:
    columns.append(column)

print(columns)

['country', "gdp(imf '20)", 'gdp per capita', 'population']


In [173]:
#transform_figures
def transform_figures(value):
    if value != 'NaN':
        split_text = value.split()
        if split_text[1] == 'Tn':
            value = float(split_text[0]) * 1000000000000
            f_value = "${:,.0f}".format(value)
            return f_value
        elif split_text[1] == 'Bn':
            value = float(split_text[0]) * 1000000000
            f_value = "${:,.0f}".format(value)
            return f_value
        elif split_text[1] == 'Mn':
            f_value = '$' + str(value)
            return f_value

    else:
        return 'NaN'
        
        

df_gdp[columns[1]] = df_gdp[columns[1]].apply(transform_figures)

In [174]:
df_gdp.head(15)

,country,gdp(imf '20),gdp per capita,population
1,United States,"$20,890,000,000,000","$61,763","338,289,857"
2,China,"$14,860,000,000,000","$10,423","1,425,887,337"
3,Japan,"$5,040,000,000,000","$40,662","123,951,692"
4,Germany,"$3,840,000,000,000","$46,100","83,369,843"
5,United Kingdom,"$2,760,000,000,000","$40,867","67,508,936"
6,India,"$2,670,000,000,000","$1,882","1,417,173,173"
7,France,"$2,620,000,000,000","$40,571","64,626,628"
8,Italy,"$1,890,000,000,000","$32,031","59,037,474"
9,Canada,"$1,650,000,000,000","$42,789","38,454,327"
10,South Korea,"$1,640,000,000,000","$31,617","51,815,810"
